# Basic DTW Code
Code from https://towardsdatascience.com/dynamic-time-warping-3933f25fcdd  
This code implements a simple DTW measure between two series represented as lists.  
It allows for series of different lengths and has a `window` parameter that determines the amount of warping allowed.  
For series of different lengths, the minimum warping will be the difference in the lengths.  

In [1]:
import numpy as np

In [2]:
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)]) # warping cannot be less than the difference in lengths. 
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix[-1,-1]

In [3]:
a = [1,2,3,3,5]
b = [1,2,2,2,2,2,2,4]
dtw(a,b, window = 3)

3.0

In [4]:
x1 = [7,7,8,9,10,10,7,4,2,1,2,4,7,11,10,9,7]
x2 = [7,8,10,10,8,7,3,2,2,4,6,12,12,9,7,7]

In [5]:
dtw_0 = dtw(x1,x2,window = 1)
dtw_0

18.0

Works also for numpy arrays.

In [6]:
x = np.array([[7,7,8,9,10,10,7,4,2,1,2,4,7,11,10,9,7],
                [7,8,10,10,8,7,3,2,2,4,6,12,12,9,7,7,8]])

In [7]:
x[0]

array([ 7,  7,  8,  9, 10, 10,  7,  4,  2,  1,  2,  4,  7, 11, 10,  9,  7])

In [8]:
for w in range(10):
    dtw_n = dtw(x[0],x[1],window = w)
    print(w, dtw_n)

0 43.0
1 19.0
2 9.0
3 9.0
4 9.0
5 9.0
6 9.0
7 9.0
8 9.0
9 9.0


In [9]:
import pandas as pd
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

#### Read the UMD_TEST.txt file using the seperator to remove white spaces. We have a shape file of (143, 151).
#### 143 rows and 151 columns.

In [10]:
# Define Dataset
umd = pd.read_csv('UMD_TEST.txt',  sep = '\s+')
umd.shape

(143, 151)

In [11]:
umd.head()

,1.0000000e+00,1.7644459e-02,3.0949268e-02,5.0555110e-02,4.4484418e-02,5.3276844e-02,4.1576200e-02,3.0947384e-02,2.7085506e-02,1.3763773e-02,...,2.4574725e-02,3.3780034e-02,2.6588896e-02,1.3932152e-02,2.4928126e-02,2.2589026e-02,3.8248000e-02,4.9837783e-02,5.3419439e-02,4.0420371e-02
0,1.0,0.041296,0.003551,0.027470,0.013158,0.009571,0.008074,0.043743,0.040592,0.012190,...,0.060539,0.046991,0.023586,0.001562,-0.002196,0.036730,0.039027,0.007754,0.004697,0.031440
1,1.0,-0.000720,0.013283,0.029450,0.045201,0.006317,0.018805,0.028901,0.013832,0.015240,...,0.016442,0.039508,0.015171,0.034708,0.010835,0.002942,0.006924,0.029502,0.040786,0.023144
2,1.0,0.005201,0.013363,0.025733,0.026653,0.038946,0.012494,0.028303,0.032011,0.009467,...,0.006383,0.037448,0.044335,0.011143,-0.003624,0.001467,0.020991,0.027675,0.001621,0.015858
3,1.0,0.022926,0.027036,0.011668,0.019500,0.036049,-0.001297,0.019717,0.039583,0.020628,...,0.026997,0.036653,0.018117,0.018314,0.012536,0.040599,0.016590,0.032730,0.002498,0.011260
4,1.0,0.004897,0.016985,0.003485,0.006491,0.028674,0.005622,0.006906,0.011857,0.022167,...,0.041374,0.030726,0.042439,0.013007,0.034942,0.026134,0.037174,0.025711,0.034479,0.028682


#### Identify the first column to pop with its values into the y variable.

In [12]:
umd.columns[0]

'1.0000000e+00'

In [13]:
y = umd.pop('1.0000000e+00').values
X = umd.values
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

In [14]:
y.shape

(143,)

In [15]:
X.shape

(143, 150)

#### In the below step we will split the dataset forming a test set that is 33% of the UMD_Test set as well at randomised in selection for the test set

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
# using the KNeighboursClassifier we include the dtw function as a Metric to use for distance computation.
kNN_3 = KNeighborsClassifier(n_neighbors=1, metric= lambda x,y: dtw(x,y,3))  
kNN_3.fit(X_train, y_train)
y_pred_3 = kNN_3.predict(X_test)

In [18]:
y_pred_3

array([3., 1., 2., 3., 3., 1., 3., 2., 2., 3., 2., 2., 2., 2., 3., 2., 1.,
       2., 3., 2., 1., 3., 3., 3., 1., 1., 1., 1., 1., 3., 2., 3., 1., 1.,
       1., 2., 3., 1., 3., 3., 3., 2., 1., 2., 1., 1., 3., 1.])

In [19]:
acc1 = kNN_3.score(X_test, y_test)
acc1

0.875

In [20]:
kNN_0 = KNeighborsClassifier(n_neighbors=1, metric= lambda x,y: dtw(x,y,0))  
kNN_0.fit(X_train, y_train)
y_pred_0 = kNN_0.predict(X_test)

In [21]:
y_pred_0

array([3., 1., 2., 3., 3., 1., 3., 2., 2., 1., 2., 2., 2., 2., 3., 2., 1.,
       2., 3., 2., 1., 3., 3., 3., 1., 1., 1., 1., 1., 3., 2., 3., 1., 1.,
       3., 2., 3., 1., 3., 2., 3., 2., 1., 2., 1., 1., 3., 2.])

In [22]:
acc0 = kNN_0.score(X_test, y_test)
acc0

0.8333333333333334

In [23]:
kNN_1 = KNeighborsClassifier(n_neighbors=1, metric= lambda x,y: dtw(x,y,1))  
kNN_1.fit(X_train, y_train)
y_pred_1 = kNN_1.predict(X_test)

In [24]:
y_pred_1

array([3., 1., 2., 3., 3., 1., 3., 2., 2., 1., 2., 2., 2., 2., 3., 2., 1.,
       2., 3., 2., 1., 3., 3., 3., 1., 1., 1., 1., 1., 3., 2., 3., 1., 1.,
       3., 2., 3., 1., 3., 2., 3., 2., 1., 2., 1., 1., 3., 2.])

In [25]:
acc1 = kNN_1.score(X_test, y_test)
acc1 

0.8333333333333334

# Hence we can see that the window value of 3 has the more accurate score in comparison to DTW with window value 0 and 1.

In [26]:
kNN_P2 = KNeighborsClassifier(n_neighbors=1, p=2, metric='minkowski')
kNN_P2.fit(X_train, y_train)
y_pred_P2 = kNN_P2.predict(X_test)
y_pred_P2

array([3., 1., 2., 3., 2., 1., 3., 2., 2., 1., 3., 2., 2., 3., 3., 2., 1.,
       2., 3., 2., 1., 3., 3., 3., 2., 1., 1., 1., 1., 3., 1., 3., 1., 1.,
       3., 2., 3., 1., 3., 3., 3., 2., 1., 2., 1., 2., 2., 2.])

In [27]:
accP2 = kNN_P2.score(X_test, y_test)
accP2 

0.9166666666666666

#### Hence we can see that the Eucledian distance ( P=2 ) not only perfroms quicker but has a higher accuracy score of 92% in comparison to the accuracy score of the best DTW option being window = 3 (and below; 2, 1 and 0) . But the DTW approach gets more accurate the larger the window size. For example at size 5 it records an accuracy score of 98%

In [28]:
kNN_5 = KNeighborsClassifier(n_neighbors=1, metric= lambda x,y: dtw(x,y,5))  
kNN_5.fit(X_train, y_train)
y_pred_5 = kNN_5.predict(X_test)
y_pred_1

array([3., 1., 2., 3., 3., 1., 3., 2., 2., 1., 2., 2., 2., 2., 3., 2., 1.,
       2., 3., 2., 1., 3., 3., 3., 1., 1., 1., 1., 1., 3., 2., 3., 1., 1.,
       3., 2., 3., 1., 3., 2., 3., 2., 1., 2., 1., 1., 3., 2.])

In [29]:
acc5 = kNN_5.score(X_test, y_test)
acc5 

0.9791666666666666